In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [2]:
# import the data file
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the name name and the EIN
charity_df = charity_df.drop(["EIN", "NAME", "CLASSIFICATION"], axis=1)

In [4]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()
charity_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [5]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [6]:
application_type_counts = charity_df.APPLICATION_TYPE.value_counts()

In [7]:
# Determine which values to replace
replace_application_type = list(application_type_counts[application_type_counts < 700].index)

# Replace in DataFrame
for app_type in replace_application_type:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app_type,"Other")


# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [8]:
income_amt_counts = charity_df.INCOME_AMT.value_counts()
income_amt_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# make a bin for 5m+
# Determine which values to replace
replace_income_amt = list(charity_df.INCOME_AMT[(charity_df.INCOME_AMT == "10M-50M") | (charity_df.INCOME_AMT == "5M-10M") | (charity_df.INCOME_AMT == "50M+")].index)
replace_income_amt

[7,
 17,
 53,
 66,
 195,
 235,
 243,
 264,
 676,
 677,
 770,
 772,
 773,
 790,
 871,
 883,
 1000,
 1001,
 1026,
 1172,
 1181,
 1208,
 1209,
 1210,
 1270,
 1278,
 1425,
 1436,
 1505,
 1512,
 1633,
 1638,
 1704,
 1713,
 1828,
 1921,
 1930,
 1972,
 2084,
 2091,
 2128,
 2165,
 2328,
 2421,
 2712,
 2782,
 2965,
 3031,
 3065,
 3121,
 3131,
 3138,
 3191,
 3263,
 3404,
 3452,
 3477,
 4348,
 4433,
 4535,
 4579,
 4659,
 4789,
 4865,
 4995,
 5059,
 5164,
 5186,
 5319,
 5519,
 5524,
 6075,
 6287,
 6417,
 6429,
 6457,
 6459,
 6465,
 6723,
 6730,
 6937,
 6990,
 7007,
 7235,
 7297,
 7299,
 7549,
 7555,
 7682,
 7849,
 7885,
 7886,
 7890,
 7891,
 7896,
 8013,
 8016,
 8191,
 8254,
 8322,
 8358,
 8457,
 8758,
 8770,
 8772,
 8813,
 8874,
 9230,
 9289,
 9293,
 9418,
 9432,
 9456,
 9866,
 9871,
 9877,
 9881,
 9908,
 9989,
 10005,
 10010,
 10039,
 10133,
 10134,
 10208,
 10211,
 10286,
 10291,
 10298,
 10306,
 10374,
 10689,
 10724,
 10776,
 10824,
 10894,
 10978,
 10985,
 10992,
 10996,
 11042,
 11126,
 111

In [10]:
# Replace in DataFrame
for income in replace_income_amt:
    charity_df.INCOME_AMT = charity_df.INCOME_AMT.replace(income,"5m+")


# Check to make sure binning was successful
charity_df.INCOME_AMT.value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(charity_df[charity_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(charity_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Merge one-hot encoded features and drop the originals
charity_df = charity_df.merge(encode_df,left_index=True, right_index=True)
charity_df = charity_df.drop(charity_cat,1)
charity_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [13]:
# Split our preprocessed data into our features and target arrays
y = charity_df["IS_SUCCESSFUL"].values
X = charity_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [14]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[ 1.39430557e-02, -3.33676580e-02, -1.55588163e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02],
       [ 1.39430557e-02,  2.86802949e-02, -1.55588163e-01, ...,
         1.43105555e+01,  2.78942470e-02, -2.78942470e-02],
       [ 1.39430557e-02, -3.34843874e-02, -1.55588163e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02],
       ...,
       [ 1.39430557e-02, -3.34843874e-02, -1.55588163e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02],
       [ 1.39430557e-02, -3.34843874e-02, -1.55588163e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02],
       [ 1.39430557e-02, -3.34843874e-02, -1.55588163e-01, ...,
        -6.98784891e-02,  2.78942470e-02, -2.78942470e-02]])

In [15]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 15
hidden_nodes_layer3 = 6

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="selu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                1110      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                465       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 16        
Total params: 1,591
Trainable params: 1,591
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Fit the model to the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Train on 25724 samples
Epoch 1/50
25724/25724 [==============================] - 3s 133us/sample - loss: 0.5902 - accuracy: 0.7140
Epoch 2/50
25724/25724 [==============================] - 3s 98us/sample - loss: 0.5728 - accuracy: 0.7257
Epoch 3/50
25724/25724 [==============================] - 3s 102us/sample - loss: 0.5709 - accuracy: 0.7259
Epoch 4/50
10240/25724 [==========>...................] - ETA: 1s - loss: 0.5674 - accuracy: 0.7271

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")